In [1]:
import sys
sys.path.append('../..')

from src.classes.tf_idf import TF_IDF
from datasets import load_from_disk
from src.classes.query import Query

KROVETZ_INDEX_LOCATION = "../../indexes/krovetz_index"
QUERY_LOCATION = "../../data/LitSearch_query"
RUN_LOCATION = "../../run_files/bm25_krovetz.run"
RUN_LOCATION2 = "../../run_files/bm25_krovetz_llm_rerank.run"
CORPUS_LOCATION = "../../data/corpus_jsonl/corpus.jsonl"

queries_full = load_from_disk(QUERY_LOCATION)['full']
queries = [q['query'] for q in queries_full]


c:\Users\keith\Desktop\Info Retrieval\irg_final_project\irg-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
index = TF_IDF(KROVETZ_INDEX_LOCATION)
dataset = load_from_disk(QUERY_LOCATION)
with open(RUN_LOCATION, 'w', encoding='utf8') as run_file:
    for (i,qline) in enumerate(dataset['full']):
        query = Query(qline['query'])
        ranking = index.search(query.get_nnn(KROVETZ_INDEX_LOCATION),'nnn',50)
        for (j,rank) in enumerate(ranking):
            run_file.write(f"{i} Q0 {rank[0]} {j+1} {rank[1]} nnn_nnn\n")

In [3]:
import os
from src.classes.llm_reranker import LLMReranker

log_path = os.path.join("../..", "logs", "nnn_nnn_krovetz.jsonl") # logs llm reranker activity


reranker = LLMReranker()  # uses OLLAMA_MODEL or defaults to "llama3.1:8b-instruct-q8_0-16k"
reranker.rerank_runfile(
    initial_run_path= RUN_LOCATION, 
    corpus_path     = CORPUS_LOCATION,
    output_run_path = RUN_LOCATION2,
    queries         = queries,  # query list
    log_path        = log_path,
    top_k=20,
    run_tag="bm_25_krovetz_llm_rerank"
)

Using Ollama model: gemma3:27b


LLM reranking: 100%|██████████| 597/597 [2:07:17<00:00, 12.79s/it]  
